# SQL Assignment

In [1]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML

In [3]:
conn = sqlite3.connect(r"C:\Users\DELL\Applied AI\Assignments\SQL\Db-IMDB-Assignment.db")

#### Overview of all tables

In [4]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
print(type(tables))
tables = tables["Table_Name"].values.tolist()
print(tables)

<class 'pandas.core.frame.DataFrame'>
['Movie', 'Genre', 'Language', 'Country', 'Location', 'M_Location', 'M_Country', 'M_Language', 'M_Genre', 'Person', 'M_Producer', 'M_Director', 'M_Cast']


In [5]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [11]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """select p.name,m.title,m.year from movie m join 
            M_Director md on m.mid = md.mid join 
            person p on md.pid = p.pid where 
            ((CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)%400 = 0) or 
            (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)%100<>0 and CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)%4=0))
            and trim(m.mid," ") in 
            (select trim(mg.mid," ") from M_Genre mg  where trim(mg.gid," ") in 
            (select trim(g.gid," ") from Genre g where g.name like '%Comedy%'));"""
grader_1(query1)

                Name                              title  year
0       Milap Zaveri                         Mastizaade  2016
1       Danny Leiner  Harold & Kumar Go to White Castle  2004
2     Anurag Kashyap                 Gangs of Wasseypur  2012
3       Frank Coraci        Around the World in 80 Days  2004
4      Griffin Dunne             The Accidental Husband  2008
5        Anurag Basu                             Barfi!  2012
6    Gurinder Chadha                  Bride & Prejudice  2004
7         Mike Judge    Beavis and Butt-Head Do America  1996
8   Tarun Mansukhani                            Dostana  2008
9       Shakun Batra                      Kapoor & Sons  2016
Wall time: 171 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [10]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results)
    assert (q2_results.shape == (17,1))

query2 = """select trim(p.name," ") from person p where trim(p.pid," ") in 
           (select trim(mc.pid," ") from M_Cast mc where trim(mc.mid," ") in 
           (select trim(m.mid," ") from movie m where m.title = 'Anand'));"""
            
grader_2(query2)

    trim(p.name," ")
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
10    Brahm Bhardwaj
11      Gurnam Singh
12      Lalita Pawar
13       Durga Khote
14        Dara Singh
15     Johnny Walker
16         Moolchand
Wall time: 201 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [7]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 ="""with pidlist(dpid) as (select trim(mc.pid," ") from M_Cast mc where trim(mc.mid," ") in 
                    (select trim(m.mid," ") from movie m where (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))<1970))
                    SELECT p.name from person p join pidlist pl on p.pid = pl.dpid; """
query_more_1990 =""" with pidlist(dpid) as (select trim(mc.pid," ") from M_Cast mc where trim(mc.mid," ") in 
                    (select trim(m.mid," ") from movie m where (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))>1990))
                    SELECT p.name from person p join pidlist pl on p.pid = pl.dpid;"""
print(grader_3a(query_less_1970, query_more_1990))


(4942, 1)
(62570, 1)
True
Wall time: 981 ms


In [6]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """ with pidlist1(dpid1) as (select trim(mc.pid," ") from M_Cast mc where trim(mc.mid," ") in 
            (select trim(m.mid," ") from movie m where (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))<1970)),
            pidlist2(dpid2) as (select trim(mc.pid," ") from M_Cast mc where trim(mc.mid," ") in 
            (select trim(m.mid," ") from movie m where (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))>1990)),
            finalpid (fpid) as (select distinct p1.dpid1 from pidlist1 p1 where p1.dpid1 in 
            (SELECT p2.dpid2 from pidlist2 p2))
            select p.name from Person p where trim(p.pid," ") in 
            (select fpid from finalpid);"""
grader_3(query3)

                Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
Wall time: 528 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [16]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a ="""SELECT md.pid,count(md.pid) film_count from m_director md where trim(md.mid," ") in 
                (select trim(m.mid," ") from movie m) group by md.pid;"""
              
print(grader_4a(query_4a))


         PID  film_count
0  nm0000180           1
1  nm0000187           1
2  nm0000229           1
3  nm0000269           1
4  nm0000386           1
5  nm0000487           2
6  nm0000965           1
7  nm0001060           1
8  nm0001162           1
9  nm0001241           1
True
Wall time: 169 ms


In [9]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """SELECT p.name,count(md.pid) film_count from person p join 
              M_Director md on p.pid = md.pid group by md.pid HAVING film_count > 9 order by film_count DESC;"""
grader_4(query4)

                    Name  film_count
0           David Dhawan          39
1           Mahesh Bhatt          35
2        Ram Gopal Varma          30
3           Priyadarshan          30
4           Vikram Bhatt          29
5   Hrishikesh Mukherjee          27
6            Yash Chopra          21
7         Shakti Samanta          19
8        Basu Chatterjee          19
9           Subhash Ghai          18
Wall time: 352 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [12]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))


query5a = """with female_cast(mid,mid_count1) as (SELECT mc.mid,count(mc.mid) FROM M_Cast mc where trim(mc.pid," ") in 
                         (SELECT trim(p.pid," ") from Person p where p.Gender = 'Female') group by mc.mid),
                  full_cast(mid,mid_count2) as (SELECT mc.mid,count(mc.mid) FROM M_Cast mc group by mc.mid)
            SELECT CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) year,count(m.year) movie_count FROM Movie m where trim(m.mid," ") in 
            (select trim(fc.mid," ") from female_cast fc where trim(fc.mid," ") in 
            (SELECT trim(fullc.mid," ") FROM full_cast fullc where fc.mid_count1 = fullc.mid_count2)) GROUP by m.year;"""
grader_5a(query5a)

   year  movie_count
0  1939            1
1  1999            1
2  2000            1
3  2018            1
Wall time: 310 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [26]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """with female_cast(mid,mid_count1) as (SELECT mc.mid,count(mc.mid) FROM M_Cast mc where trim(mc.pid," ") in 
                                                 (SELECT trim(p.pid," ") from Person p where p.Gender = 'Female') group by mc.mid),
            full_cast(mid,mid_count2) as (SELECT mc.mid,count(mc.mid) FROM M_Cast mc group by mc.mid),
            table1(year,movie_count) as (SELECT CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) year,count(CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)) FROM Movie m where trim(m.mid," ") in 
                                        (select trim(fc.mid," ") from female_cast fc where trim(fc.mid," ") in 
                                        (SELECT trim(fullc.mid," ") FROM full_cast fullc where fc.mid_count1 = fullc.mid_count2)) GROUP by CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))
        SELECT table1.year,round(((table1.movie_count*100.0)/count(CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))),2) female_movies_percentage,count(CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)) movies_in_year from movie m join 
        table1 on CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) = table1.year GROUP by CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER);"""
grader_5b(query5b)

   year  female_movies_percentage  movies_in_year
0  1939                     50.00               2
1  1999                      1.52              66
2  2000                      1.56              64
3  2018                      0.96             104
Wall time: 691 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [25]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """with onecast(dmid,dpid) as (SELECT DISTINCT mc.mid,mc.pid from M_Cast mc)
            select m.title,count(oc.dmid) cast_size from movie m join 
            onecast oc on trim(m.mid," ") = trim(oc.dmid," ")
            group by oc.dmid order by cast_size desc;"""
grader_6(query6)

                        title  cast_size
0               Ocean's Eight        238
1                    Apaharan        233
2                        Gold        215
3             My Name Is Khan        213
4  Captain America: Civil War        191
5                    Geostorm        170
6                     Striker        165
7                        2012        154
8                      Pixels        144
9       Yamla Pagla Deewana 2        140
Wall time: 2min 57s


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [8]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """with count_table(year,movie_count) as (select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER),
                                       count(CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)) movie_count from movie m 
                                       group by CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) 
                                       order by CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER))
             select year,sum(movie_count) over (order by year range BETWEEN current row and 9 following) as decade_sum 
             FROM count_table order by decade_sum desc;"""
grader_7a(query7a)


   year  decade_sum
0  2008        1203
1  2009        1200
2  2007        1186
3  2005        1168
4  2006        1158
5  2004        1145
6  2003        1112
7  2010        1091
8  2002        1089
9  2001        1046
Wall time: 46.9 ms


In [14]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """with count_table(year,movie_count) as (select CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER),
                                       count(CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)) movie_count from movie m 
                                       group by CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) 
                                       order by CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)),
             decade(top_decade,decade_movie_count) as (select year,sum(movie_count) over (order by year range BETWEEN current row and 9 following) as decade_sum 
             FROM count_table order by decade_sum desc)
             select * from decade where decade_movie_count>=(select max(decade_movie_count) from decade);"""
grader_7(query7)


   top_decade  decade_movie_count
0        2008                1203
Wall time: 15.6 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [11]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """SELECT trim(mc.pid," ") actor,trim(md.pid," ") director,count(*) movies_in_combo FROM M_Cast mc 
            join  M_Director md on trim(mc.mid," ") = trim(md.mid," ") GROUP by actor,director;"""
grader_8a(query8a)


       actor   director  movies_in_combo
0  nm0000002  nm0496746                1
1  nm0000027  nm0000180                1
2  nm0000039  nm0896533                1
3  nm0000042  nm0896533                1
4  nm0000047  nm0004292                1
5  nm0000073  nm0485943                1
6  nm0000076  nm0000229                1
7  nm0000092  nm0178997                1
8  nm0000093  nm0000269                1
9  nm0000096  nm0113819                1
Wall time: 2min 47s


In [8]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245,2))

query8 = """with combo_count(actor,director,movies_in_combo) as (SELECT trim(mc.pid," ") actor,trim(md.pid," ") director,
                                               count(*) movies_in_combo FROM M_Cast mc join  M_Director md on 
                                               trim(mc.mid," ") = trim(md.mid," ") GROUP by actor,director),
            max_combo(actor,director,max_in_combo) as (SELECT actor,director,max(movies_in_combo) max_in_combo 
                                          from combo_count group by actor),
            final(actor,movies_with_yash) as (select mc.actor,cc.movies_in_combo from max_combo mc join 
                                             combo_count cc on cc.actor = mc.actor 
              where cc.director = (select trim(p.pid," ") FROM Person p where trim(p.name," ") like "Yash%Chopra") 
              and cc.movies_in_combo = mc.max_in_combo)
    select p.name,f.movies_with_yash from Person p join final f on trim(p.pid," ") = f.actor 
    order by f.movies_with_yash desc;"""
                                        
grader_8(query8)

                Name  movies_with_yash
0        Jagdish Raj                11
1   Manmohan Krishna                10
2           Iftekhar                 9
3      Shashi Kapoor                 7
4     Waheeda Rehman                 5
5      Rakhee Gulzar                 5
6     Achala Sachdev                 4
7        Neetu Singh                 4
8           Ravikant                 4
9    Parikshat Sahni                 3
(245, 2)
Wall time: 6min 26s


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [9]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """with smid as (select trim(mc.mid," ") mid from M_Cast mc where trim(mc.pid," ") in 
              (SELECT trim(p.pid," ") from person p where trim(p.name," ") = "Shah Rukh Khan"))
            select DISTINCT mc.pid from M_Cast mc 
            where trim(mc.pid," ") <> (SELECT trim(p.pid," ") from person p where trim(p.name," ") = "Shah Rukh Khan") 
            and trim(mc.mid," ") in (SELECT mid from smid);"""
grader_9a(query9a)


          PID
0   nm0004418
1   nm1995953
2   nm2778261
3   nm0631373
4   nm0241935
5   nm0792116
6   nm1300111
7   nm0196375
8   nm1464837
9   nm2868019
(2382, 1)
Wall time: 751 ms


In [10]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """with smid as (select trim(mc.mid," ") mid from M_Cast mc where trim(mc.pid," ") in 
              (SELECT trim(p.pid," ") from person p where trim(p.name," ") = "Shah Rukh Khan")),
            s1 as (select DISTINCT trim(mc.pid," ") s1_pids from M_Cast mc 
            where trim(mc.pid," ") <> (SELECT trim(p.pid," ") from person p where trim(p.name," ") = "Shah Rukh Khan") 
            and trim(mc.mid," ") in 
            (SELECT mid from smid)),
            s2_movies as (select distinct trim(mc.mid," ") s2_mids from M_Cast mc where trim(mc.mid," ") not in 
              (SELECT mid from smid) and trim(mc.pid," ") in 
              (select s1_pids from s1)),
            s2 as (select DISTINCT trim(mc.pid," ") s2_pids from M_Cast mc where trim(mc.pid," ") not in 
            (select s1_pids from s1) and trim(mc.mid," ") in 
            (SELECT s2_mids from s2_movies))
            select trim(p.name) from Person p where trim(p.pid," ") in 
            (select s2_pids from s2);"""
grader_9(query9)

            trim(p.name)
0           Freida Pinto
1            Rohan Chand
2           Damian Young
3        Waris Ahluwalia
4  Caroline Christl Long
5          Rajeev Pahuja
6      Michelle Santiago
7        Alicia Vikander
8           Dominic West
9         Walton Goggins
(25698, 1)
Wall time: 816 ms
